**PySDS Week 3 Lecture 1. V.1**
Last author: B. Hogan

# Interfacing with the web - Part I (Working with Reddit)

The World Wide Web is a collection of interlinked resources accessible through URIs. It is a protocol / data layer over the Internet. The documents are a form of hyper-text, meaning that the text itself contains links to other text. 

Social Data is not necessarily web data or internet data. Yet, the open web has been such a success that interfacing with social data tends to involve some form of online data.

For the purposes of the class here, we need to know a few foundational things about the web in order to retrieve data from it programmatically. WE will cover argument strings and then move to an example with authentication in Reddit. This will be a sustained example that will show a variety of features. 

## Web requests: GET and POST. 

If you want to get a server to send your computer data , such as a web page, you have to request that data from the server (literally a machine that serves up documents). If the document doesn't care who you are, you can typically use a GET request, since it is simple. You ask for the data at a web address and it returns the data. Now the "web address" itself might be a very complex address. But we might still treat it as an address. A POST request, by contrast, sends up data with the address as a 'payload'. 

Side note: I've enjoyed the presentation of the differences between these two at https://www.diffen.com/difference/GET-vs-POST-HTTP-Requests. But some of those differences are just FYI, but some of them relate to security. The reason we need to know them is because when dealing with APIs, we often have to use the right one or else we cannot authenticate ourselves to a server and thus get the correct data. Below we will use a GET request and then we will skip post in favor of using a 'wrapper'. 



## Get Requests - Anatomy of a URL 

A URL (or more generally a URI; Uniform Resource Identifier) is an address used to refer to a specific set of documents or data. We can se URL / URI interchangably, but strictly speaking, URL is a subclass of a URI dedicated to the web. 

~~~
[Transfer  [Domain         [Sub           [File]   [Argument
protocol]   name]          domain]                  strings]
http://    www.reddit.com /r/learnpython  .json    ?after=t3_a4r234 
~~~

Notes: 
- **Transfer Protocol**: How are we transferring the file? Some common protocols: HTTP, HTTPS, FTP, SFTP. All of these tend to run on Internet Protocol addresses (IP addresses). 
- **Domain name**: The human-readable root page address. This is registered in a naming service so that when you look it up it returns the machine-relevant Internet Protocol address (IP address). Most addresses are IPV4, and take four digits  separated by periods. IPV6 is a very long address designed to ensure there are enough IP addresses for the multitude of possible connections through the 'Internet of Things'. These are your google.com and yandex.ru addresses. 
- **Top-level Domain**: The part after the last period in a domain name. For BBC.co.uk it is UK.
- **Domain name system Servers [DNS servers]**: Look up tables for returning the IP address when given a name. These help translate between the name that you submit for as the URL and the IP address. 
- **Argument strings**: After the end of a web address you can include arguments. These are formatted as key-value pairs. They are appended to the URL using a ? For example in the URL: 
~~~
https://www.google.com/search?q=argumenttext
~~~
There is one argument, the key is the letter 'q' (which stands for query) and the value is "argumenttext". 

So a get request will send a URL and some arguents up to a server and receive some data for that. It is what you are probably used to. 

To employ a URL string there are a couple important features to bear in mind. 
- Text has to be "quoted". So searching for "First name" in a browser window actually searches for "First%20name".  
- The program that sends the request also sends other information along with it including the IP address and the User-Agent string.
- **User-agent strings**: Each browser / client (such as a python program ;) has a set user agent string that identifies the client. This is for at least two reasons:
  - To see unique computers on a network. 
  - To format text or the page accordingly. Ever wonder how a page gives you the 'mac' or the 'windows' download? It is because it reads the user-agent string. There are other ways, particularly using javascript to get this information from the user, but the UA is often simple /  good enough. 

Let's see what the web can learn about your browser. 

https://www.whoishostingthis.com/tools/user-agent/

It learned a lot of information using javascript. To prevent this information  being learned we can surf without it (but we will typically have a bad time without javascript). 

To work with URLs in python we can use the URL library or ```urllib```. In the example below we will download the OII's mainpage and then print out the first 1024 characters. (Notice what happens when we remove the ```.decode("utf-8")``` ).

In [ ]:
import urllib.request 

URL = "http://www.oii.ox.ac.uk/"
req = urllib.request.Request( URL, headers={'User-Agent': 'OII SDS class 2018.1/Hogan'})
infile = urllib.request.urlopen(req)

print(infile.read(500).decode("utf-8")) #500 here refers to the first 500 characters. 
                                        # Printing all of it seems excessive.


Now as you recall, we have been looking into using data from the web. We will not be scraping that html page there, but it is certainly possible. We can use a number of strategies from raw downloading with urllib to crawling with pseudo-browsers such as ```Selenium``` and ```Mechanize```. We will be discussing this more later. For now lets look at downloading reddit data in .json format. 

In [ ]:
import urllib.request, json 

URL = "http://www.reddit.com/r/eyebleach.json"

req = urllib.request.Request( URL, headers={'User-Agent': 'OII SDS class 2018.1/Hogan'})

infile = urllib.request.urlopen(req)
reddit_json = json.loads(infile.read())
print(reddit_json.keys())

As can be seen, reddit loads in as json, where the top element is a dictionry with two keys, 'kind' and 'data'. We can navigate through this dictionry to see what it is we want to parse. Notice that below we can preview the 'after' tag. That tag can help us navigate through the content. This is because reddit ccontent is feed-based, like Twitter, and thus always changing. We can't simply ask for post 1 or 3 because those numbers are only meaningful from a specific time frame. 

In [ ]:
print(reddit_json.keys() )
print(reddit_json['data'].keys() )
print(reddit_json['data']['after'])

## Creating the argument string.

Using Reddit as an example, we can see some of the uses of URL strings for querying dynamic content. In particular, it helps to collect a specific snapshot of a set of content that, typical of social media, is always changing. 

As Reddit has an infinitely scrolling list of content that changes dynamically, there are a variety of ways to think about collecting this content. The same can be said for tweets or newsfeed items. In reddit's case, the standard is to give you content in their preferred 'top' order. This order is determined by an algorithm that, loosely-speaking, sorts content based on log transformed values of voting over time. But this order then will change dynamically over time. How to page through this document? We can see this in the .json. Surfing to http://api.reddit.com/ you can see that it immediately comes down in json. The root has two keys, 'kind' and 'data'. Data has several keys inside. The one with content is ```children```, but there are two other important keys: ```before``` and ```after```. To page through the document you can use the 'after' key with your url in order to page throught the document. We will demonstrate this below. 

How would you know this without looking through the JSON file? The Reddit documentation. We should check there now to get a sense of how this should work. We will then implement a way to page through multiple reddit documents, creating a function that helps out. We will also see that reddit has OAuth for authentication. We will examine that after the paging example. 

The documentation is at https://www.reddit.com/dev/api 

From there we will also want to look at Reddit's access rules. 

https://github.com/reddit-archive/reddit/wiki/API


Notice the document said a few things: 
1. You will be rate limited to 60 queries per minute. So perhaps we should have a ```time.sleep(1)``` method.
2. Reddit throttles generic user-agent strings and has a specific formate with which it expects strings. 
3. When paging you must use the before OR after string, but it is recommended to also include count. 
    
See all three of these put together in the code below: 

In [ ]:
# Lots of import statements here at the top. 

from pandas.io.json import json_normalize
import urllib.request 
import pandas as pd

import time 
import json
import math 

In [ ]:
# I have put the bulk of the work in a single function. 
# it has a few arguments at the top. Now, its also reasonable to split this
# into two functions. The first manages the downloads and the second does the 
# specific downloading 
def redditReader (page="http://api.reddit.com/",max_items=25,user_agent='',subreddit=None): 
    ''' Downloading data from reddit
    
    keyword arguments: 
    
    '''
    
    if user_agent == '':
        user_agent ={'user-agent':"'User-Agent': windows:com.demo.sdsclass2018:v1 (by /u/berniehogan)"}
    if subreddit:
        page += "r/%s" %subreddit
    
    frame_list = [] #Add each json frame to a list, concat all later.
    count = 0
    for i in range(math.ceil(max_items / 25)):
        print(i)
        time.sleep(1)
        req = urllib.request.Request( page, headers=user_agent)
        infile = urllib.request.urlopen(req)
        reddit_json = json.loads(infile.read())
        
        frame_list.append(json_normalize(reddit_json["data"]["children"]))
        print(len(frame_list[i]))
        page = page.split("?")[0] + "?after=%s&count=%s" % (reddit_json["data"]["after"],count)
        count += 25

    return pd.concat(frame_list)

result = redditReader(max_items=100,subreddit="aww")

In [ ]:
result.columns

Sometimes Reddit has stickied posts that stay at the top of the page. We can remove these using the following: 

In [ ]:
result = result[result['data.stickied'] != True]
len(result)

Slight diversion: sometimes when concatenating it gets cross when you don't ensure that your columns are in the right order. See the example below for the reasons for the issue.

In [ ]:
# Sort alignment warning for future versions: 
import pandas as pd 

df1 = pd.DataFrame({"a": [1, 2], "b": [3, 4]}, columns=['b', 'a'])
df2 = pd.DataFrame({"b": [5, 6]})

print (pd.concat([df1, df2], sort=True),end="\n\n")

# Notice if you don't sort you have to explicitly say so. In the future, B might get added to the #
# bottom of A because it is labelled 'b' but it isn't the right column. 
print (pd.concat([df1, df2], sort=False))


In [ ]:
# len(result["data.title"].unique())
result["data.title"].value_counts()

# Installing praw 

We are going instal an external package now. Why? Because sometimes we want to do something that's not available in Anaconda. In this case, we are going to install the Python Reddit API Wrapper, or praw. This package will manage the act of logging in to reddit so that we can access some of the APIs that we couldn't otherwise. To note, to do this, you will need to have a reddit account. Fortunately, you can create one easily without even an email account. 

There are a number of ways to install packages and some might be trouble if you don't do it right. We will stick with 'conda', the package manager for anaconda. The second way is using 'pip'. You can do these either through the workbook or the terminal / anaconda prompt. 

See here for a good summary if how to install and why to be careful:
https://jakevdp.github.io/blog/2017/12/05/installing-python-packages-from-jupyter/

The doucmentation is here:
https://praw.readthedocs.io/en/latest/tutorials/comments.html

In [ ]:
try: 
    import praw
except ModuleNotFoundError:
    import sys
    !{sys.executable} -m pip install praw
    import praw

## OAUTH and Keys 

OAUTH is a form of authentication is used to manage user credentials. We will see this again tomorrow with Twitter, in a slightly different form of OAUTH. In order to get a key we have to first register a reddit name and store a password. In general, OAUTH credentials don't want a password, but this will involve the use of a web request and a third party server. You can see this in the PRAW documentation under ["Web Application"](https://praw.readthedocs.io/en/latest/getting_started/authentication.html#web-application). 

The user name and password aren't, strictly speaking, necessary for credentials, but they are necessary in order to get a series of tokens. Typically OAUTH has public tokens and secret tokens. The public token is used to identify the user, the secret token is used to confirm that the user is who they say they are using 'public key encryption'. It is for 'signing' requests, and this signature is read by the server. https://www.reddit.com/prefs/apps/

OAUTH2 is somewhat different in that it does not need the user's credentials at all, but instead gives the user unique access tokens through an intermediary (like a Facebook pop-up dialogue). Let's get our keys and fill them in below. 

In [ ]:
1/0

keys = {"client_id":"", 
        "client_secret":"", 
        "username":"berniehogan", 
        "password":""}

with open("keys.json",'w') as infile:
    infile.write( json.dumps(keys) )

Once you have filled in those values, cleared the 1/0 and run the code, you should have all the details stored in a .json file in the same directory as your code. Now, you can delete the cell above and simply read the keys.json file as needed. Below, we will load in the keys file and then use it to create a ```reddit``` instance. 

In [ ]:
keys = json.loads(open("keys.json").read())

reddit = praw.Reddit(user_agent='Comment Extraction (by /u/berniehogan)',
                     client_id=keys['client_id'], 
                     client_secret=keys['client_secret'],
                     username=keys['username'], 
                     password=keys['password'])

In [ ]:
# Let's check that it worked. 
# If it did it will print my name.
# If it didn't it will throw an error. 
print(reddit.user.me())

Now that we have confirmed that we are indeed logged in and have a user account token through OAuth, we can start to interact with the reddit API as a series of python objects, rather than simply as a set of unstructured .json. Now note that this is very tricky, as sometimes data comes down in ways you might not expect. It is good practice to check for errors, test your code and read the documenntation. 

## Analysing a reddit AMA: Frank OZ of the muppets

Approxmately nine months ago Frank Oz, the puppeteer behind Fozzie Bear and  Miss Piggy did an AMA on Reddit. AMAs are "Ask Me Anything", and often hosted by the reddit administration themselves. To query for the reddit submission, we can create a ```submission``` object. These can be created using the full URL that can be found in the browser, or mire lightweight - the Reddit ID for the content. These IDs are categorically tagged sequential objects. The categories are "t1" through "t6", 
- t1_	Comment
- t2_	Account
- t3_	Link
- t4_	Message
- t5_	Subreddit
- t6_	Award

A submission is a t3 object, whereas a user is a t2 object. Be careful, because sometimes the object has the category attached. It's not clear when it does or doesn't have this prefix. 


In [ ]:
subr = reddit.subreddit('aww')

In [ ]:
nba = reddit.subreddit('nba')
posts = nba.hot(limit=200)
for i in posts: 
    print(i.title)
    dir(i)
#     print(i.authorname)
    break

In [ ]:
# Let's inspect a submission 
submission = reddit.submission(id='7o6bxv')
for i in dir(submission): print(i)

The submisssion has a great deal of metadata in there. This is different from the data that we got from the link on the submission page (seen in the DataFrame above). These objects are not all static names. Some of them are nested objects in their own right. Each submission has a number of comments, each comment has an author, each author has their own details. This can all be downloaded by navigating the object. 

We will be looking at comments below. By getting the first comment, we are not querying the full comment tree. 

In [ ]:
comment = submission.comments.list()[0]
print(dir(comment))
print()
print(dir(comment.author))

As we can see, there's a lot of info for both the comment and the author. We are going to build a small dataframe for all the comments in this submission. Then we can ask some questions of the comments. One of these questions will be done with just this dataframe, but one of them will require us to build a tree structure to represent the comment tree. 
```
<submission> 
    └<root_comment>
        └<child_comment>
            └<child_comment>
    └<root_comment>
        └<child_comment>
        └<child_comment>
            └<child_comment>
        └<child_comment>
```
This comment tree is known as a DAG, a directed, acyclic graph. The edges are from the children comments to the parent comment. 

We can use the DAG to ask some questions about the overall structure of the comment tree. We don't need to ask about the whole tree, but it is interesting as a means for understanding the dynamics on reddit. This follows the work of https://www.ingentaconnect.com/content/pal/paljit/2010/00000025/00000002/art00011

who classified forums on slashdot according to their thread depth and thread width. Shallow, wide threads were associated with gaming and genaral leisure. Long, narrow threads were associated with politics. Yet, interesting operating systems had a structural trace that was similar. Back in 2010 it seemed Mac vs. Windows was in full force. 

Before building a tree, however, we should first build a DataFrame of all the comments. We can then use this DataFrame to build a DAG. What should we put in our DataFrame? I've inspected several elements and I think the ones below are interesting: 
(note that author.name might be missing, as is the case with deleted content) This might spell trouble later on when comment.author.name returns None. Consequently, none is replaced with ```[deleted]```
    

In [ ]:
print(comment.body)
print(comment.ups)
print(comment.author.name)
print(comment.created_utc)
print(comment.depth)
print(comment.is_root)

Below we will get all the comments. This is part of PRAW. With ```limit=none``` it will go through all the comment tree, one at a time, while observing the API query limits (i.e. one query per second), so it might take a couple minutes to get all the comments for this particular thread.  

In [ ]:
submission.comments.replace_more(limit=None)
print("Got all the comments")
print(len( submission.comments.list()  )  )

Let's put them all into a data frame. Not all the data, mind you, but the data that we think is interesting.

In [ ]:
df_list = []
flag = True 
for c,comment in enumerate(submission.comments.list()):
    # THe try exists because some reddit comments are from authors who 
    # have deleted their account, but the comments persist. 
    try:
        x = comment.author.name,
        authorname = x[0]
    except AttributeError:
        authorname = "[deleted]"

    try:
        comment_body = comment.body,
    except AttributeError:
        comment_body = "[deleted]"

    try: 
        df_list.append([ \
        comment.id,
        comment_body,
        authorname,
        comment.ups,
        comment.created_utc,
        comment.depth,
        comment.parent_id[3:]
        ])
    except AttributeError:
        if flag: 
            print("Are you sure you collected all the comments first?")
            flag = False
        
reddit_df = pd.DataFrame(df_list,columns=["id","body","authorname","ups","created_utc","depth","parent_id"])
display(reddit_df.head(20))

Just with the comment data, we can do some interesting things. For exampple, we can convert the UTC timestap to a datetime object in the DataFrame and then plot the upvotes overtime. Again, back to map and lambda. 

In [ ]:
from datetime import datetime 

reddit_df["utc"] = reddit_df["created_utc"].map(lambda x: datetime.utcfromtimestamp(x) )
reddit_df["zero_utc"] = reddit_df["created_utc"] - reddit_df["created_utc"].min()

In [ ]:
%matplotlib inline
reddit_df.plot.scatter("zero_utc","ups")

## Creating a DAG. 

This is not a course in network analysis, but it does not need to be to get some of the very basics down. First, a network or graph has nodes (in this case comments) and edges (in this case, the 'in reply to' structure). A reply to a reply to a reply has a depth of 3.
- What is the deepest comment thread? 
- What comment prompted the most replies? 

First we build a network, then we take some measurements of the network, then we can merge this back with the main data frame and start looking at some of the properties of the comments. Later you might wish to ask questions such as, do wide comments get more upvotes than narrow ones? Is there any difference between comments based on sentiment? What does the largest comment tree look like? 

First we import ```networkx```, one of the two leading python network programs. The other ```igraph``` is  real nuisance to install and the graphic package for it is downright formidable to install. It has nice clustering features though, but that's for another course. 

Second, we go through the nodes twice, the first time creating a new node for each comment in the file. The second time, we add the links between all of them. You could do this in one go, but the issue is that we wanted to add the ```name``` and the ```depth``` to the node before building the links. 



In [ ]:
import networkx as nx 
import pandas as pd

G = nx.DiGraph()

node_set = set([])
# submission.comments.replace_more(limit=None)
for i in submission.comments.list():
    try:
        G.add_node(i.id, depth=i.depth,name=i.author.name)
    except AttributeError: 
        G.add_node(i.id, depth=i.depth,name="[Deleted]")
    node_set.add(i.id)

for i in submission.comments.list():
    if i.parent_id and i.parent_id[3:] in node_set:
        G.add_edge(i.id,i.parent_id[3:])

# pd.Series([G.subgraph(c).number_of_nodes() for c in nx.connected_components(G)]).plot('hist')

print(G.number_of_nodes())

### Laying out the graph 

This is some boilerplate code and more for illustrative purposes. As you can see, it does some pretty standard steps - layout the nodes according to an algorithm, draw nodes, draw edges, display plot. In this case, it is the Kamada Kawai algorithm for layout. There are others that might work better but this one is available. The only thing that might not be obvious is the 'weakly_connected_component_subgraphs' method. This method looks for the connected subgraphs. Each root comment is its own subgraph that includes all of the root node's children. We want to have a look at the largest one of these. So we take the 'max' of the set of the weakly connected components. 

In [ ]:
%pylab inline
import matplotlib.pyplot as plt 

# # Determine the layout.
Gc = max(nx.weakly_connected_component_subgraphs(G), key=len)
pos=nx.kamada_kawai_layout(Gc)

# Drawing the nodes, and setcting the size. 
nx.draw_networkx_nodes(Gc,pos,node_size=10)

# Drawing the edges, and setting the width
nx.draw_networkx_edges(Gc,pos)

# Drawing some labels 
# nx.draw_networkx_labels(g, pos)

plt.axis('off')
# plt.show()

Below here we are using the concept of "in-degree", which is just a count of the number of edges that point to a node. A high in-degree means lots of replies. Does a root node get the most replies? Turns out no, it is a comment of depth 2 that gets a whopping 29 replies, far outpacing the next highest.

The code below is a bit denser than usual, perhaps. We can unpack it by taking little bits of it at a time and learning what it does. For example, we notice that G.in_degree doesn't return a dict, but a dict-like object that we must turn into a dictioary. Then we can group by depth. Here instead of taking the ```mean``` or the ```count``` as is often the case, we will take the maximum in-degree. However, two nodes at the same depth might both have the same indegree. We take both of them, then. 

In [ ]:
count = 0
for i,j in G.in_degree():
    print(i,j)
    count += 1
    if count > 10: break

We can see that we have a comment ID as the ID of the node, and the in degree as the value. Thus we can merge this back into our original DataFrame now. 

In [ ]:
deg_dict = pd.DataFrame.from_dict (  dict(G.in_degree()), orient="index",columns=["in_degree"])

In [ ]:
deg_dict['depth'] = pd.Series(  
    dict([ (i,G.nodes[i]['depth']) for i in G.nodes] ) ) 


In [ ]:
# 
df_list = []
for i,j in deg_dict.groupby("depth"):
    df_list.append(j[j["in_degree"] == max(j["in_degree"])].copy())

In [ ]:
df_list

In [ ]:
max_replies_per_depth = pd.concat(df_list)
display(max_replies_per_depth)

Now that we have the comments, we can merge them back into the main DataFrame and then look at the longest comments. In this case, I made a new, smaller dataframe with just the comments that qualified as the widest at each depth. We can see the one that had 29 replies. It is, interestingly, not from Fzank Oz 

In [ ]:
z = max_replies_per_depth.merge(reddit_df,left_index=True, right_on="id")
display(z)

In [ ]:
z['body'] = z['body'].map(lambda max_replies_per_depth:max_replies_per_depth[0])
z

In [ ]:
print(z.loc[616,"body"])

Finally for today, we can read the longest thread in the entire comment set as its own by using networkx to query for the longest path and then uerying the DataFrame for those comments. In theory, there should really only be one entry at each depth, so we can sort by depth and then print out the comments. 

In [ ]:
longest_path_comment_branch = list(nx.dag_longest_path(G))

In [ ]:
longest_path_comment_branch

In [ ]:
lpcb_df =reddit_df[reddit_df["id"].isin(longest_path_comment_branch)]

In [ ]:
lpcb_df

In [ ]:
temp_sorted = lpcb_df.sort_values("depth",ascending=True)

for index,row in temp_sorted.iterrows():
    print("%s said:\n%s\n---\n" % (row.authorname,row.body))